In [2]:
!pip install jupyter-dash



Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Run the app in JupyterLab on a different port
app.run_server(port=8051, mode='inline')


In [ ]:
# Import required libraries
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output

# Load SpaceX Launch dataset
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard"),
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'}] +
                     [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
                 value='ALL',
                 placeholder="Select a Launch Site",
                 searchable=True),
    html.Br(),

    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    dcc.RangeSlider(id='payload-slider',
                    min=0, 
                    max=10000, 
                    step=1000,
                    marks={i: str(i) for i in range(0, 10001, 2000)},
                    value=[min(spacex_df['Payload Mass (kg)']), max(spacex_df['Payload Mass (kg)'])]),
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])

# Define callback to update pie-chart based on selected site
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value'))
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches by Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(filtered_df, names='class', title=f"Launch Successes for site {selected_site}")
    return fig

# Define callback to update scatter-chart based on selected site and payload range
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')])
def update_scatter_chart(selected_site, payload_range):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color="Booster Version Category",
                     title="Correlation between Payload and Success for selected site(s)")
    return fig

# Run the app in JupyterLab on a different port
app.run_server(port=8051, mode='inline')
